In [31]:
import pandas as pd
import numpy as np 
import torch 
import cv2
import glob
import os
from torchvision.models import resnet50, ResNet50_Weights
import torch.nn as nn 

In [32]:
p1 = "/Users/menghang/Desktop/ml-dev/ml-replicate/vector-db/facial_recognition/lfw_funneled/Alejandro_Toledo"
p2 = "/Users/menghang/Desktop/ml-dev/ml-replicate/vector-db/facial_recognition/lfw_funneled/Allyson_Felix"

In [33]:
p1_embed ={
    "genre": "men", 
    "data": []
}
p2_embed ={
    "genre": "women", 
    "data": []
}

resnet = resnet50( weights=ResNet50_Weights.IMAGENET1K_V2)
model = nn.Sequential(*list(resnet.children())[:-1])

In [34]:
for img_path in sorted(glob.glob(p1 + "/*"), key=os.path.getctime): 
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224,224)).transpose((2,0,1))
    features = model(torch.Tensor([img]))
    flattened = features.view(features.size(0), -1)[0].tolist()
    p1_embed['data'].append(flattened)


In [35]:
for img_path in sorted(glob.glob(p2 + "/*"), key=os.path.getctime): 
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224,224)).transpose((2,0,1))
    features = model(torch.Tensor([img]))
    flattened = features.view(features.size(0), -1)[0].tolist()
    p2_embed['data'].append(flattened)


In [53]:
import chromadb 
client = chromadb.Client()
collection = client.get_or_create_collection("facial-recognition",metadata={"hnsw:space": "cosine"},)


In [47]:
collection.add(embeddings=p2_embed["data"], metadatas=[{"gender": "female"} for _ in range(len(p2_embed["data"]))] , ids=[f"id-f{x}" for x in range(len(p2_embed["data"]))])

Add of existing embedding ID: id-f0
Add of existing embedding ID: id-f1
Add of existing embedding ID: id-f2
Add of existing embedding ID: id-f3
Add of existing embedding ID: id-f4
Insert of existing embedding ID: id-f0
Insert of existing embedding ID: id-f1
Insert of existing embedding ID: id-f2
Insert of existing embedding ID: id-f3
Insert of existing embedding ID: id-f4


In [54]:
collection.add(embeddings=p1_embed["data"], metadatas=[{"gender": "male"} for _ in range(len(p1_embed["data"]))] , ids=[f"id-m{x}" for x in range(len(p1_embed["data"]))])

Add of existing embedding ID: id-m0
Add of existing embedding ID: id-m1
Add of existing embedding ID: id-m2
Add of existing embedding ID: id-m3
Add of existing embedding ID: id-m4
Add of existing embedding ID: id-m5
Add of existing embedding ID: id-m6
Add of existing embedding ID: id-m7
Add of existing embedding ID: id-m8
Add of existing embedding ID: id-m9
Add of existing embedding ID: id-m10
Add of existing embedding ID: id-m11
Add of existing embedding ID: id-m12
Add of existing embedding ID: id-m13
Add of existing embedding ID: id-m14
Add of existing embedding ID: id-m15
Add of existing embedding ID: id-m16
Add of existing embedding ID: id-m17
Add of existing embedding ID: id-m18
Add of existing embedding ID: id-m19
Add of existing embedding ID: id-m20
Add of existing embedding ID: id-m21
Add of existing embedding ID: id-m22
Add of existing embedding ID: id-m23
Add of existing embedding ID: id-m24
Add of existing embedding ID: id-m25
Add of existing embedding ID: id-m26
Add of exis

In [55]:
img = cv2.imread("/Users/menghang/Desktop/ml-dev/ml-replicate/vector-db/Alejandro_Toledo_0037.jpg")
img = cv2.resize(img, (224,224)).transpose((2,0,1))
features = model(torch.Tensor([img]))
flattened = features.view(features.size(0), -1)[0].tolist()

In [56]:
results = collection.query(query_embeddings=flattened, n_results=10)

In [57]:
results

{'ids': [['id-m17',
   'id-m29',
   'id-m11',
   'id-m16',
   'id-m36',
   'id-m38',
   'id-m20',
   'id-m28',
   'id-m9',
   'id-f3']],
 'distances': [[13.239620208740234,
   14.626680374145508,
   15.118236541748047,
   15.335177421569824,
   15.775787353515625,
   15.857386589050293,
   15.976330757141113,
   17.17392921447754,
   17.39162254333496,
   17.69011878967285]],
 'metadatas': [[{'gender': 'male'},
   {'gender': 'male'},
   {'gender': 'male'},
   {'gender': 'male'},
   {'gender': 'male'},
   {'gender': 'male'},
   {'gender': 'male'},
   {'gender': 'male'},
   {'gender': 'male'},
   {'gender': 'female'}]],
 'embeddings': None,
 'documents': [[None, None, None, None, None, None, None, None, None, None]],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}